# Web scraping

這個notebook的重點是從幾個網路對應的位址抓取資料  
核心的資料撈取部分以TISV的data為主，這邊會展示如何使用以封裝好的function  
資料量繁多，後續為了求能快速整理與應用，只有使用ETag相關的資料  
VD資料量太大且需再做更進一步清整，因此僅附上如何撈取
其他參考可以取得的參考資訊附在末端，實際應用時只有取部分的資料來應用

# lib

In [30]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import time
import sys

# 將 hwttp 資料夾加入 Python 搜尋路徑
sys.path.append(os.path.abspath('..'))
import hwttp.data_scraper as ds

# TISVCloud Data Scraping 

資料獲取的方式已經封裝在.py之中了，這邊只需要call幾個function就可以完成資料抓取  
在封裝的python file中設計大概是1~1.5s 左右對指定的時間範圍進行單檔下載，如果會不斷遭遇到錯誤，建議進到python file中去調整sleep的時間  
原則上不要壓到1s內作業，否則會給官方的server帶來壓力，下載作業會直接被reject，更嚴重會被鎖ip  

以下會簡易做幾個步驟的示範:
1. 首先是抓取資料的時間範圍設定，會利用簡單的list與str完成範圍  
2. 再來call抓取資料的function即可(所有資料抓取的function都是如此)  

In [28]:
date_list = []
start_date = '20231101'
end_date = '20231102'
date_list = ds.generate_date_strings(start_date, end_date)

## VD data(volume)

VD data的部分，可以有條件的選擇下載靜態或是動態的資料(也可以全部)    
靜態資料一天只會有1筆，動態則是每分鐘會產出1筆(1天1440筆)  

使用的funciton中，有以下幾個input，示範中沒有引入的部分是因為有預設，如果需要異動可以透過傳入參數處理  

date_list: list[str], 輸入日期串，像是['20230101', '20230102']  
save_dir_upper: str, 資料爬取存放的上層路徑，其下日期的子資料夾會自動生成  
select_data: str, 原始資料有區分靜態、動態，'both'= 全數下載, 'static'=下載靜態, 'dynamic'= 下載動態  

In [33]:
ds.scrape_vd_data(date_list, select_data='all')

Directory '../data/raw/VD/' created.


100%|██████████| 2/2 [1:03:52<00:00, 1916.20s/it]


In [34]:
ds.scrape_vd_data(date_list, select_data='static')

Directory '../data/raw/VD/' already exists.


100%|██████████| 2/2 [00:02<00:00,  1.38s/it]


## Etag info data

這部分資料的原始路徑如以下所示  
eTag 靜態資訊(v2.0) = ./history/motc20/ETag/YYYYMMDD/ETag_0000.xml.gz  
eTag 配對路徑靜態資訊(v2.0) = ./history/motc20/ETag/YYYYMMDD/ETagPair_0000.xml.gz  
eTag 配對路徑動態資訊(v2.0) = ./history/motc20/ETag/YYYYMMDD/ETagPairLive_HHmm.xml.gz  

使用的funciton中，有以下幾個input，示範中沒有引入的部分是因為有預設，如果需要異動可以透過傳入參數處理  

date_list: list[str], 輸入日期串，像是['20230101', '20230102']  
save_dir_upper: str, 資料爬取存放的上層路徑，其下日期的子資料夾會自動生成  
select_data: str, 原始資料有區分靜態、動態，'all'= 全數下載, 'info'=靜態資訊下載, 'pair_static'= 配對靜態下載, 'pair_dynamic' = 配對動態下載 

In [35]:
ds.scrape_etag_info(date_list, select_data='all')

  0%|          | 0/2 [00:00<?, ?it/s]

Directory '../data/raw/ETag/20231101' created.


 50%|█████     | 1/2 [05:14<05:14, 314.35s/it]

Directory '../data/raw/ETag/20231102' created.


100%|██████████| 2/2 [08:38<00:00, 259.12s/it]


In [39]:
ds.scrape_etag_info(date_list, select_data='pair_static')

 50%|█████     | 1/2 [00:00<00:00,  6.84it/s]

Directory '../data/raw/ETag/20231101' already exists.
Directory '../data/raw/ETag/20231102' already exists.


100%|██████████| 2/2 [00:00<00:00,  4.39it/s]


## ETag intergantry Travel Time

可指定一連串的時間資訊下進行ETag gantry間通過所花費的旅行時間為主資料的爬取  

date_list: list[str], 輸入日期串，像是['20230101', '20230102']  
save_dir: str, 資料下載後儲存的路徑，如果不存在會自動建立folder  

In [40]:
ds.scrape_etag_intergantry_traveltime(date_list)

Directory '../data/raw/ETag_intergantry_traveltime' created.


100%|██████████| 2/2 [00:02<00:00,  1.49s/it]


## ETag Gantry Volume

可指定一連串的時間資訊下進行ETag gantry通過車流量為主資料的爬取  

date_list: list[str], 輸入日期串，像是['20230101', '20230102']  
save_dir: str, 資料下載後儲存的路徑，如果不存在會自動建立folder  

In [41]:
ds.scrape_etag_gantry_volume(date_list)

Directory '../data/raw/ETag_gantry_vol' created.


100%|██████████| 2/2 [00:02<00:00,  1.47s/it]


## ETag intergantry Speed

可指定一連串的時間資訊下進行ETag gantry間通過車速為主資料的爬取  

date_list: list[str], 輸入日期串，像是['20230101', '20230102']  
save_dir: str, 資料下載後儲存的路徑，如果不存在會自動建立folder  

In [42]:
ds.scrape_etag_intergantry_speed(date_list)

Directory '../data/raw/ETag_intergantry_speed' created.


100%|██████████| 2/2 [00:03<00:00,  1.71s/it]


# Extra Data Scraping

## 中油資訊(oil price)

In [10]:
# 透過url取得來取得網頁
url = "https://vipmbr.cpc.com.tw/mbwebs/showhistoryprice_oil.aspx"
response = requests.get(url)
response.encoding = 'utf-8'

# 解析網頁內容
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'id': 'MyGridView'})

# 從中提取表格數據
rows = table.find_all('tr')
header = [th.text.strip() for th in rows[0].find_all('th')]

data = []
for row in rows[1:]: # 需要跳過第1 row
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols])  # 去掉空值

# 建立dataframe好額外儲存
df = pd.DataFrame(data, columns=header)
df.to_csv('../data/raw/oil_prices.csv', index=False)

In [11]:
df

,調價日期,無鉛汽油92,無鉛汽油95,無鉛汽油98,超級/高級柴油,二行程無鉛,煤油,甲種漁船油(KL),乙種漁船油(KL),低硫燃料油(0.5%)(KL),甲種低硫燃料油/低硫鍋爐油(0.5%)(KL),特種低硫燃料油(KL),低硫鍋爐油(1%)(KL)
0,2024/9/23,29.3,30.8,32.8,28,,,20793,16878,,,,
1,2024/9/17,,,,,,39,,,,,,
2,2024/9/16,29,30.5,32.5,27.6,,,20393,16478,,,,
3,2024/9/10,,,,,,39.7,,,,,,
4,2024/9/9,29.5,31,33,28.2,,,20993,17078,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,2020/1/22,,,,,,35.1,,,,,,
442,2020/1/20,27.1,28.6,30.6,25,,,17793,12878,,,,
443,2020/1/15,,,,,,37.1,,,,,,
444,2020/1/13,27.8,29.3,31.3,25.7,,,18493,13578,,,,


## 壅塞路段整理

壅塞路段資訊部分，為pdf文件，這邊暫無撰寫抓取與處理方式  
"https://1968.freeway.gov.tw/storage/pdf/112年12月國道易壅塞路段彙整表.pdf"  
需要使用的話可以針對url直接替換其中的a年b月來下載對應月份的資料  
語法上會是 f"https://1968.freeway.gov.tw/storage/pdf/{a}年{b}月國道易壅塞路段彙整表.pdf"  
資料應用上是直接手動建立資料，針對我需要的地點跟時間等寫一個簡單的csv做記錄   